In [14]:

"""
BACKTESTING FROM PREVIOUS PROJECT

MOVING AVERAGE CROSSOVER STRATEGY USING THE S&P 500 AND 3 YEARS OF DATA. 

"""

import numpy as np
import pandas as pd
# import hvplot.pandas
from pathlib import Path
import alpaca_trade_api as tradeapi
import requests
import os 
from dotenv import load_dotenv
# import plotly.express as px
# import panel as pn
# import hvplot
# import hvplot.pandas

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import datetime, timedelta


# pn.extension()

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

pd.set_option("display.max_rows", None, "display.max_columns", None)

%matplotlib inline

In [15]:
#load alpaca keys

load_dotenv()

alpaca_public = os.getenv("ALPACA_API_KEY")
alpaca_secret = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_public, alpaca_secret, api_version='v2')


type(alpaca_public)

In [13]:
#Use alpha vantage to download 20 years of data for the our selected ticker. 

ticker = 'SPY'

stock_data_df = api.alpha_vantage.historic_quotes(ticker, adjusted=True, output_format='pandas')
stock_data_df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2020-08-21,477.050,499.472,477.0000,497.48,497.48,84513660.0,0.0,1.0
2020-08-20,463.000,473.568,462.9335,473.10,473.10,31726797.0,0.0,1.0
2020-08-19,463.933,468.650,462.4400,462.83,462.83,36384502.0,0.0,1.0
2020-08-18,457.410,464.000,456.0300,462.25,462.25,26408385.0,0.0,1.0
2020-08-17,464.250,464.350,455.8501,458.43,458.43,29431414.0,0.0,1.0


In [29]:
#Clean data

#Sort earliest to latest.
stock_data_df.sort_index(inplace=True, ascending=True)

# Drop nulls
stock_data_df.dropna(inplace=True)

# drop duplicates
stock_data_df.drop_duplicates(inplace=True)

#count nulls 
stock_data_df.isnull().sum()

stock_data_df.head()



,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
1999-11-01,136.5000,137.0000,135.5625,135.5625,91.9725,4006500.0,0.0,1.0
1999-11-02,135.9687,137.2500,134.5937,134.5937,91.3152,6516900.0,0.0,1.0
1999-11-03,136.0000,136.3750,135.1250,135.5000,91.9301,7222300.0,0.0,1.0
1999-11-04,136.7500,137.3593,135.7656,136.5312,92.6297,7907500.0,0.0,1.0
1999-11-05,138.6250,139.1093,136.7812,137.8750,93.5414,7431500.0,0.0,1.0


In [30]:
#Set up stock data for backtesting, needs to be OHLCV

stock_data_df.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
stock_data_df.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
stock_data_df.head()
#stock_data_df.to_csv('stock_data.csv')


,Open,High,Low,Close,Volume
date,,,,,
1999-11-01,136.5000,137.0000,135.5625,91.9725,4006500.0
1999-11-02,135.9687,137.2500,134.5937,91.3152,6516900.0
1999-11-03,136.0000,136.3750,135.1250,91.9301,7222300.0
1999-11-04,136.7500,137.3593,135.7656,92.6297,7907500.0
1999-11-05,138.6250,139.1093,136.7812,93.5414,7431500.0


In [31]:

#  #Slice out recent bear market
# stock_data_df.drop(stock_data_df.tail(94).index,inplace=True)
# stock_data_df.tail()

 #Slice out last 3 years of data
stock_data_df = stock_data_df.iloc[-750:]

#stock_data_df = stock_data_df.iloc[:200]
stock_data_df.tail()

,Open,High,Low,Close,Volume
date,,,,,
2020-08-13,336.61,338.2514,335.8300,336.83,41816146.0
2020-08-14,336.41,337.4200,335.6200,336.84,47260390.0
2020-08-17,337.94,338.3400,336.8517,337.91,34496002.0
2020-08-18,338.34,339.1000,336.6100,338.64,38733908.0
2020-08-19,339.05,339.6100,336.6200,337.23,68054244.0


In [32]:
len(stock_data_df)

750

In [33]:

#Function to do a backtest using Backtesting library. This function will find the best SMA cross strategy by analyzing
#different combinations of long short smas.

#Change the ranges in the stats for different scenarios. For example, range(5,365,5) means to test smas 
#between 5 and 365 days and use a 5 day step. 

def backtest():
    class SmaCross(Strategy):
        
    #n1 =short, n2 = long, they can be anything at the start of the optimization   
        n1=1
        n2=200
        
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close,self.n1)
            self.ma2 = self.I(SMA, Close,self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()

        
                
    bt_stock = Backtest(stock_data_df, SmaCross,
                              cash=10000, commission=.000)
  
    stats = bt_stock.optimize(n1=range(1, 200, 1),
                    n2=range(1, 200, 1),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2)
   

    bt_stock.run()
    #bt_spy_plot = bt_stock.plot()
    #return bt_stock_plot
    return stats
    #return bt_stock.run()
backtest()


C:\Users\costa\Anaconda3\envs\vizenv\lib\site-packages\ipykernel_launcher.py:33: UserWarning:

Searching best of 19701 configurations.



Start                      2017-08-28 00:00:00
End                        2020-08-19 00:00:00
Duration                    1087 days 00:00:00
Exposure [%]                           90.0644
Equity Final [$]                       20624.5
Equity Peak [$]                        20710.7
Return [%]                             106.245
Buy & Hold Return [%]                  46.1537
Max. Drawdown [%]                     -16.1643
Avg. Drawdown [%]                     -5.23998
Max. Drawdown Duration       258 days 00:00:00
Avg. Drawdown Duration        32 days 00:00:00
# Trades                                    65
Win Rate [%]                           49.2308
Best Trade [%]                         20.8769
Worst Trade [%]                       -5.61663
Avg. Trade [%]                         1.07549
Max. Trade Duration           72 days 00:00:00
Avg. Trade Duration           16 days 00:00:00
Expectancy [%]                         2.34404
SQN                                    1.92209
Sharpe Ratio 

In [34]:
# To confirm the above results we will do our own tests. 

#To set up the crossover strategy select the one column we need, "Close", and set to dataframe
signals_df = stock_data_df['Close'].to_frame()

signals_df.head()


,Close
date,
2017-08-28,230.7365
2017-08-29,231.0007
2017-08-30,232.0951
2017-08-31,233.4914
2017-09-01,233.8216


In [35]:
#Generate signals and put in dataframe

# Set the short window and long windows
short_window = 2
long_window = 13

#Set names of the windows
short_name = "SMA2"
long_name = "SMA13"

# Generate the short and long moving averages 
signals_df[short_name] = signals_df["Close"].rolling(window=short_window).mean()
signals_df[long_name] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the short SMA is under the long SMA, and
# where 1 is when the short SMA is higher (or crosses over) the long SMA
signals_df["Signal"][short_window:] = np.where(
    signals_df[short_name][short_window:] > signals_df[long_name][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Buy/Sell"] = signals_df["Signal"].diff()

signals_df.to_csv('signals_2_13.csv') 
signals_df.head()


,Close,SMA2,SMA13,Signal,Buy/Sell
date,,,,,
2017-08-28,230.7365,NaN,NaN,0.0,NaN
2017-08-29,231.0007,230.86860,NaN,0.0,0.0
2017-08-30,232.0951,231.54790,NaN,0.0,0.0
2017-08-31,233.4914,232.79325,NaN,0.0,0.0
2017-09-01,233.8216,233.65650,NaN,0.0,0.0


In [36]:
len(signals_df)

750

In [38]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Buy/Sell'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Buy/Sell'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# # Visualize close price for the investment
# security_close = signals_df[['Close']].hvplot(
#     line_color='lightgray',
#     ylabel='Price in $',
#     width=1000,
#     height=400
# )
# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='black',
    ylabel='Price in $',
    width=1000,
    height=400
)
# Visualize moving averages
moving_avgs = signals_df[['SMA2', 'SMA13']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (Close)
   .Scatter.Close.II :Scatter   [date]   (Close)

In [39]:
signals_df.head()

,Close,SMA2,SMA13,Signal,Buy/Sell
date,,,,,
2017-08-28,230.7365,NaN,NaN,0.0,NaN
2017-08-29,231.0007,230.86860,NaN,0.0,0.0
2017-08-30,232.0951,231.54790,NaN,0.0,0.0
2017-08-31,233.4914,232.79325,NaN,0.0,0.0
2017-09-01,233.8216,233.65650,NaN,0.0,0.0


In [16]:
# Add share and cash columns

signals_df["Shares"] = float(0)
signals_df["Portfolio Cash"] = float(100000)

#Set up starting cash variable for sma strategy and buy and hold strategy

cash = 100000
buy_and_hold_cash = 100000


#Iterate through each row, use combinations of signal and buy/sell points to set shares and portfolio cash for each row.

for index, row in signals_df.iterrows():
    
    #If at a buy point, calculate the number of shares using total cash available and set the cash to zero
    
    if row['Buy/Sell'] == 1.0:
       
        shares = cash/row['Close']
        row['Shares'] = shares
        row['Portfolio Cash']= 0 
       
    #If at sell point, calculate the cash generated from the sale (shares X closing price) and set the shares to zero, then
    #update the cash variable with the new cash balance'''
    
    elif row['Buy/Sell'] == -1.0:
        row['Portfolio Cash']= shares*row['Close']
        cash = row['Portfolio Cash']
        row['Shares']= 0
        
    #If we are not at a buy or sell point, look to signal column, if the signal is buy, then set cash to zero and update shares.
    # If the signal is sell, update the cash column using cash variable
    else:
     
        row['Buy/Sell'] == 0
        if row['Signal']== 1:
            row['Portfolio Cash'] = 0
            row["Shares"]= shares
        elif row['Signal'] == 0:
            row['Portfolio Cash'] = cash
        
# Add a Portfolio Total column
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + (signals_df['Shares']*signals_df['Close'])

# Calculate how many shares a buy and hold investor could buy at the start of the comparison. Use end of long window as start. 
buy_and_hold_shares = buy_and_hold_cash/signals_df['Close'].iloc[long_window-1]

# Add a buy and hold total column for comparison, set the first 225 (long window) to 100000 so both portfolios are at 100000 at the
#end of the long window calculation
signals_df["Buy&Hold Total"]= buy_and_hold_shares * signals_df['Close']
signals_df["Buy&Hold Total"][0:long_window-1] = buy_and_hold_cash      
        
# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()  
signals_df['Buy&Hold Daily Returns']= signals_df['Buy&Hold Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cum Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1
signals_df['Buy&Hold Cum Returns'] = (1 + signals_df['Buy&Hold Daily Returns']).cumprod() - 1

# Print the DataFrame
pd.set_option("display.max_rows", None, "display.max_columns", None)
signals_df.round(2)
signals_df.tail(50)

,Close,SMA2,SMA13,Signal,Buy/Sell,Shares,Portfolio Cash,Portfolio Total,Buy&Hold Total,Portfolio Daily Returns,Buy&Hold Daily Returns,Portfolio Cum Returns,Buy&Hold Cum Returns
date,,,,,,,,,,,,,
2020-04-27,285.7850,283.75395,278.613631,1.0,0.0,539.330710,0.000000,154132.626838,283135.979773,0.014419,0.014419,0.541326,1.831360
2020-04-28,284.4708,285.12790,279.509669,1.0,0.0,539.330710,0.000000,153423.838419,281833.961457,-0.004599,-0.004599,0.534238,1.818340
2020-04-29,291.9178,288.19430,280.659192,1.0,0.0,539.330710,0.000000,157440.234214,289211.933153,0.026178,0.026178,0.574402,1.892119
2020-04-30,289.1999,290.55885,281.794169,1.0,0.0,539.330710,0.000000,155974.387278,286519.226120,-0.009310,-0.009310,0.559744,1.865192
2020-05-01,281.5437,285.37180,281.717585,1.0,0.0,539.330710,0.000000,151845.163499,278933.993556,-0.026474,-0.026474,0.518452,1.789340
2020-05-04,282.3203,281.93200,282.162538,0.0,-1.0,0.000000,152264.007728,152264.007728,279703.395036,0.002758,0.002758,0.522640,1.797034
2020-05-05,284.9288,283.62455,282.705523,1.0,1.0,534.393181,0.000000,152264.007728,282287.716128,0.000000,0.009240,0.522640,1.822877
2020-05-06,282.9973,283.96305,282.522485,1.0,0.0,534.393181,0.000000,151231.827300,280374.119736,-0.006779,-0.006779,0.512318,1.803741
2020-05-07,286.4122,284.70475,282.988885,1.0,0.0,534.393181,0.000000,153056.726573,283757.366083,0.012067,0.012067,0.530567,1.837574


In [17]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Buy/Sell'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Buy/Sell'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)

In [18]:
#Plot the SMA Strategy vs the Buy & Hold strategy

signals_df[["Buy&Hold Total", "Portfolio Total"]].hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [18]:
#Prepare evaluation metrics dataframe

# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest', 'Buy&Hold']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` 
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,NaN,NaN
Cumulative Returns,NaN,NaN
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [19]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'][0] = signals_df['Portfolio Cum Returns'][-1]
portfolio_evaluation_df.loc['Cumulative Returns'][1] = signals_df['Buy&Hold Cum Returns'][-1]

portfolio_evaluation_df




,Backtest,Buy&Hold
Annual Return,NaN,NaN
Cumulative Returns,0.558969,2.14072
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [20]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'][0] = (signals_df['Portfolio Daily Returns'].mean() * 252)
portfolio_evaluation_df.loc['Annual Return'][1] = (signals_df['Buy&Hold Daily Returns'].mean() * 252)


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0290281,0.0768673
Cumulative Returns,0.558969,2.14072
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [21]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'][0] = (signals_df['Portfolio Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df.loc['Annual Volatility'][1] = (signals_df['Buy&Hold Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0290281,0.0768673
Cumulative Returns,0.558969,2.14072
Annual Volatility,0.11635,0.197599
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [22]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'][0] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df.loc['Sharpe Ratio'][1] = (
    signals_df['Buy&Hold Daily Returns'].mean() * 252) / (
    signals_df['Buy&Hold Daily Returns'].std() * np.sqrt(252)
)


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0290281,0.0768673
Cumulative Returns,0.558969,2.14072
Annual Volatility,0.11635,0.197599
Sharpe Ratio,0.249489,0.389007
Sortino Ratio,NaN,NaN


In [23]:
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

sortino_ratio_df2 = signals_df[['Buy&Hold Daily Returns']].copy()
sortino_ratio_df2.loc[:,'Downside Returns1'] = 0

target = 0
mask = sortino_ratio_df2['Buy&Hold Daily Returns'] < target
sortino_ratio_df2.loc[mask, 'Downside Returns1'] = sortino_ratio_df2['Buy&Hold Daily Returns']**2


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.0290281,0.0768673
Cumulative Returns,0.558969,2.14072
Annual Volatility,0.11635,0.197599
Sharpe Ratio,0.249489,0.389007
Sortino Ratio,NaN,NaN


In [24]:
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'][0] = sortino_ratio

down_stdev = np.sqrt(sortino_ratio_df2['Downside Returns1'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df2['Buy&Hold Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'][1] = sortino_ratio


In [25]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [76]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Buy/Sell'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Shares']*row['Close']
        share_size = row['Shares']
        entry_share_price = row['Close']

    elif row['Buy/Sell'] == -1:
        exit_date = index
        exit_portfolio_holding = row['Portfolio Total']
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df
                
                

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,SPY,2016-07-11,2016-09-20,506.812831,197.3115,198.3307,100000.000000,100516.543638,516.543638
1,SPY,2016-09-29,2018-12-06,503.838283,199.5016,260.4992,100516.543638,131249.469701,30732.926063
2,SPY,2018-12-24,2019-06-11,576.725040,227.5772,281.7925,131249.469701,162516.790744,31267.321044


In [77]:
#Create plots

price_df = signals_df[['Close', 'SMA220', 'SMA230']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [78]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest,Buy&Hold]

In [79]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

In [80]:
#Build dashboard with panel

# Create rows
price_chart_row = pn.Row(price_chart)
portfolio_evaluation_row = pn.Row(portfolio_evaluation_table)
trade_evaluation_row = pn.Row(trade_evaluation_table)

# Create columns
portfolio_column = pn.Column('# Portfolio Evaluation Metrics', price_chart_row, portfolio_evaluation_row)
trade_column = pn.Column('# Trade Evaluation Metrics', trade_evaluation_row)

# Create tabs
trading_dashboard = pn.Tabs(
    ("Portfolio Metrics", portfolio_column),
    ("Trade Metrics", trade_column)
)

In [81]:
trading_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(NdOverlay)
        [2] Row
            [0] HoloViews(Table)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Table)